## 1. Setup and Installation

First, ensure you have the required dependencies installed and set up your environment.

In [ ]:
# Import required modules
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Add the project root to the path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Set OpenAI API key (replace with your key or use environment variable)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

print(f"Project root: {project_root}")
print(f"API key configured: {bool(os.environ.get('OPENAI_API_KEY'))}")

In [ ]:
# Import the LLM provider components
from src.llm import ProviderFactory, OpenAIProvider
from src.prompts.builder import MSAPromptBuilder, HARPromptBuilder
from src.core.models import LLMRequest, LLMResponse

print("✓ Imports successful!")

## 2. Building Prompts for RTEC Rule Generation

The prompt builders create comprehensive prompts that include:
- RTEC base knowledge (predicates, fluents, etc.)
- Domain-specific knowledge (events, fluents, background knowledge)
- Examples of rule generation
- The specific activity description

### 2.1 Maritime Situational Awareness (MSA) Domain

In [ ]:
# Create MSA prompt builder
msa_builder = MSAPromptBuilder()

# See available MSA activities
print("Available MSA activities:")
for activity in list(msa_builder.activity_map.keys())[:5]:  # Show first 5
    print(f"  - {activity}")
print(f"  ... and {len(msa_builder.activity_map) - 5} more")

In [ ]:
# Build initial prompt for "gap" activity
messages = msa_builder.build_initial("gap")

print(f"Number of messages: {len(messages)}")
print(f"\nMessage 1 (System): {messages[0]['role']}")
print(f"Length: {len(messages[0]['content'])} characters")
print(f"\nFirst 500 characters of system message:")
print(messages[0]['content'][:500] + "...")

print(f"\n\nMessage 2 (User): {messages[1]['role']}")
print(f"\nUser message (activity description):")
print(messages[1]['content'])

### 2.2 Human Activity Recognition (HAR) Domain

In [ ]:
# Create HAR prompt builder
har_builder = HARPromptBuilder()

# See available HAR activities
print("Available HAR activities:")
for activity in har_builder.activity_map.keys():
    print(f"  - {activity}")

In [ ]:
# Build initial prompt for "leaving_object" activity
har_messages = har_builder.build_initial("leaving_object")

print(f"Number of messages: {len(har_messages)}")
print(f"\nMessage 1 (System): {har_messages[0]['role']}")
print(f"Length: {len(har_messages[0]['content'])} characters")
print(f"\nFirst 500 characters of system message:")
print(har_messages[0]['content'][:500] + "...")
print(f"\n\nMessage 2 (User): {har_messages[1]['role']}")
print(f"\nUser message (activity description):")
print(har_messages[1]['content'])

### 2.3 Controlling Example Types

You can specify whether to include simple fluent examples, static fluent examples, or both.

In [ ]:
# Get different types of examples
simple_examples = msa_builder.get_examples(fluent_type="simple")
static_examples = msa_builder.get_examples(fluent_type="static")
all_examples = msa_builder.get_examples(fluent_type="both")

print(f"Simple fluent examples: {len(simple_examples)}")
print(f"Static fluent examples: {len(static_examples)}")
print(f"All examples: {len(all_examples)}")

# Note: By default, build_initial() uses fluent_type="both"

### 2.4 Building Prompts with Specific Example Types

You can control which examples are included when building prompts by specifying the `fluent_type` parameter.

In [ ]:
# Build prompts with only simple fluent examples
messages_simple = msa_builder.build_initial("gap", fluent_type="simple")
print(f"Prompt with SIMPLE examples only:")
print(f"  System message length: {len(messages_simple[0]['content'])} characters")
print(f"  Number of examples: {len(msa_builder.get_examples('simple'))}")

# Build prompts with only static fluent examples
messages_static = msa_builder.build_initial("gap", fluent_type="static")
print(f"\nPrompt with STATIC examples only:")
print(f"  System message length: {len(messages_static[0]['content'])} characters")
print(f"  Number of examples: {len(msa_builder.get_examples('static'))}")

# Build prompts with both types (default)
messages_both = msa_builder.build_initial("gap", fluent_type="both")
print(f"\nPrompt with BOTH example types (default):")
print(f"  System message length: {len(messages_both[0]['content'])} characters")
print(f"  Number of examples: {len(msa_builder.get_examples('both'))}")

print("\n" + "="*80)
print("Why this matters:")
print("="*80)
print("• SIMPLE fluent examples show initiatedAt/terminatedAt patterns")
print("• STATIC fluent examples show holdsFor patterns")
print("• Choose based on the type of activity you're defining")
print("• Using 'both' provides the LLM with the most context")

## 3. Using the LLM Provider

The provider abstraction makes it easy to work with different LLM APIs through a unified interface.

### 3.1 Creating a Provider with the Factory

In [ ]:
# Create an OpenAI provider using the factory
provider = ProviderFactory.create(
    provider_name="openai",
    api_key=os.environ.get('OPENAI_API_KEY'),
    # Optional: add additional configuration
    # timeout=60,
    # max_retries=3,
)

print(f"Provider created: {type(provider).__name__}")
print(f"Available providers: {ProviderFactory.list_providers()}")

### 3.2 Generating RTEC Rules from Messages

Now we can send our constructed messages to the LLM to generate RTEC rules.

In [ ]:
# Generate rules for the "gap" activity
# Note: This will make an actual API call to OpenAI

response = provider.generate_from_messages(
    messages=messages,
    model="gpt-4",  # or "gpt-3.5-turbo" for faster/cheaper results
    temperature=0.3,  # Lower temperature for more deterministic output
    max_tokens=2000,
)

print(f"Response ID: {response.request_id}")
print(f"Provider: {response.provider}")
print(f"Model: {response.model}")
print(f"Tokens used: {response.tokens_used}")
print(f"Latency: {response.latency_ms:.2f}ms")
print(f"Finish reason: {response.finish_reason}")
print(f"\n{'='*80}")
print("Generated RTEC Rules:")
print(f"{'='*80}")
print(response.content)

### 3.3 Alternative: Using LLMRequest Model

In [ ]:
# You can also create an LLMRequest object
# This is useful when you have a single prompt string (for example a finetuned model or a 
# model that will use RAG to fetch relevant content.)

request = LLMRequest(
    provider="openai",
    model="gpt-4",
    prompt="Generate RTEC rules for a vessel loitering activity.",
    temperature=0.5,
    max_tokens=1500,
    metadata={"activity": "loitering", "domain": "MSA"},
)

# Note: generate() internally converts the prompt to messages format
response = provider.generate(request)
print(response.content)

print("LLMRequest created (not executed to save API calls)")
print(f"Model: {request.model}")
print(f"Temperature: {request.temperature}")
print(f"Metadata: {request.metadata}")